In [1]:
!pip install -r ../requirements.txt

  Using cached torch-2.0.1-cp311-cp311-win_amd64.whl (172.3 MB)
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/00/45/ec3407adf6f6b5bf867a4462b2b0af27597a26bd3cd6e2534cb6ab029938/filelock-3.12.2-py3-none-any.whl.metadata
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Using cached networkx-3.1-py3-none-any.whl (2.1 MB)
  Using cached Jinja2-3.1.2-py3-none-any.whl (133 kB)
  Obtaining dependency information for MarkupSafe>=2.0 from https://files.pythonhosted.org/packages/be/bb/08b85bc194034efbf572e70c3951549c8eca0ada25363afc154386b5390a/MarkupSafe-2.1.3-cp311-cp311-win_amd64.whl.metadata
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
Using cached filelock-3.12.2-py3-none-any.whl (10 kB)
Using cached MarkupSafe-2.1.3-cp311-cp311-win_amd64.whl (17 kB)


In [2]:
# import cell
import json
import pandas as pd
from tqdm import tqdm
import sys, os

In [3]:
my_dir = sys.path[0]
sys.path.append(os.path.join(my_dir, '..'))
from eli5 import *

In [13]:
valid_path = '../docs/ELI5.jsonl'

In [14]:
valid_dataset = ELI5(valid_path, img_size, transform=transform)

NameError: name 'ELI5' is not defined

In [8]:
with open('../docs/ELI5.jsonl', 'r') as json_file:
    json_list = list(json_file)

In [11]:
a = json.loads(json_list[0])

In [16]:
for i in a:
    print(f'{i} --> {a[i]}')

question_id --> 1oy5tc
question --> in football whats the point of wasting the first two plays with a rush - up the middle - not regular rush plays i get those
answers --> ["Keep the defense honest, get a feel for the pass rush, open up the passing game. An offense that's too one dimensional will fail. And those rushes up the middle can be busted wide open sometimes for big yardage.", "If you throw the ball all the time, then the defense will adapt to always cover for a pass.  By doing a simple running play every now and then, you force the defense to stay close and guard against the run.  Sometimes, the offense can catch the defense off guard by faking a run and freeing up their receivers.\n\nAlso, you don't have to gain massive yards on every single play.  Sometimes, it works best to gain a few yards at a time.  As long as you get the first down, you are in good shape.", 'In most cases the O-Line is supposed to make a hole for the running back to go through. If you run too many plays

In [3]:
train = pd.read_json(path_or_buf="../docs/ELI5.jsonl", lines=True)
val = pd.read_json(path_or_buf="../docs/ELI5_val.jsonl", lines=True)

ValueError: Could not reserve memory block

In [ ]:
print(train[0])